In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import tensorflow as tf

np.set_printoptions(linewidth=100, precision=4)
sess = tf.InteractiveSession()

batch_size = 3
num_rois = 32
num_classes = 4
num_detections = 100

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Manipulate 3D Tensors

In [75]:
mrcnn_class = tf.Variable(tf.random_uniform([batch_size, num_rois, num_classes], minval=0, maxval=1  , dtype=tf.float32, seed=1234), name = 'var')
bbox_4d     = tf.Variable(tf.random_uniform([batch_size, num_rois, num_classes, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')
output_rois = tf.Variable(tf.random_uniform([batch_size, num_rois, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')

one   = tf.ones([num_rois, 4])
test_bboxes = tf.stack([one, one+1, one+2, one+3], axis = 0) 
# fill8_3d = tf.zeros([3,8,4], dtype = tf.float32)

init = tf.global_variables_initializer().run()

###  Create class score tensor
 batch_size x  num_dectections x num_classes

In [76]:
# print(' padding required for mrcnn_class : ', padding.eval())
print('  Scores .shape', mrcnn_class.get_shape() )
print(mrcnn_class.eval())

  Scores .shape (3, 32, 4)
[[[0.8483 0.3236 0.3067 0.0697]
  [0.9139 0.1705 0.2834 0.3563]
  [0.5416 0.0753 0.0745 0.866 ]
  [0.5591 0.0093 0.067  0.4977]
  [0.2989 0.8094 0.6585 0.4815]
  [0.1517 0.591  0.3344 0.1552]
  [0.2455 0.1395 0.9869 0.9031]
  [0.1522 0.023  0.7943 0.6682]
  [0.7623 0.9574 0.8136 0.2078]
  [0.8659 0.1014 0.2265 0.0317]
  [0.1861 0.4301 0.1205 0.2435]
  [0.3574 0.1943 0.192  0.2904]
  [0.8358 0.9281 0.8308 0.0583]
  [0.3079 0.9126 0.3954 0.7867]
  [0.3893 0.0964 0.6395 0.6672]
  [0.7801 0.5502 0.5758 0.4603]
  [0.5407 0.5492 0.2173 0.6376]
  [0.482  0.0279 0.954  0.9278]
  [0.7984 0.7105 0.8229 0.7032]
  [0.4165 0.0972 0.9167 0.4561]
  [0.1692 0.3054 0.6161 0.5193]
  [0.6092 0.147  0.124  0.0996]
  [0.3477 0.3669 0.3179 0.4279]
  [0.9821 0.3264 0.8513 0.0144]
  [0.7459 0.5724 0.9563 0.6972]
  [0.1117 0.7005 0.66   0.7265]
  [0.3661 0.5429 0.7805 0.6744]
  [0.2175 0.2784 0.5751 0.1126]
  [0.9919 0.7484 0.8062 0.1836]
  [0.9662 0.0838 0.7904 0.0944]
  [0.9445 0.4

###   Get max prediction score and  corresponding class for each ROI

The the highest score for each detection (highest score amongst classes) and the class corresponding to the highest score 

In [77]:
pred_classes = tf.to_int32(tf.argmax(mrcnn_class,axis=-1))
pred_classes_exp = tf.to_float(tf.expand_dims(pred_classes,axis=-1))
print(' classes with highest scores:', pred_classes.get_shape())
print(pred_classes.eval())

print(mrcnn_class.get_shape())
pred_scores  = tf.reduce_max(mrcnn_class,axis=-1, keep_dims=True)
print(' highest scores:', pred_scores .get_shape())

# pred_scores _exp = tf.expand_dims(pred_scores , axis=-1)   # (32,)
# print(' highest scores:', pred_scores _exp.get_shape())
print(' pred_scores    \n', pred_scores .eval())

 classes with highest scores: (3, 32)
[[0 0 3 0 1 1 2 2 1 0 1 0 1 1 3 0 3 2 2 2 2 0 3 0 2 3 2 2 0 0 0 3]
 [1 1 3 3 0 3 2 0 3 3 3 3 3 3 1 3 0 2 3 1 0 0 1 2 0 0 3 3 0 0 1 3]
 [1 3 3 0 3 0 2 3 1 0 0 1 1 1 3 3 2 3 0 3 3 2 2 0 3 2 3 3 0 2 1 0]]
(3, 32, 4)
 highest scores: (3, 32, 1)
 pred_scores    
 [[[0.8483]
  [0.9139]
  [0.866 ]
  [0.5591]
  [0.8094]
  [0.591 ]
  [0.9869]
  [0.7943]
  [0.9574]
  [0.8659]
  [0.4301]
  [0.3574]
  [0.9281]
  [0.9126]
  [0.6672]
  [0.7801]
  [0.6376]
  [0.954 ]
  [0.8229]
  [0.9167]
  [0.6161]
  [0.6092]
  [0.4279]
  [0.9821]
  [0.9563]
  [0.7265]
  [0.7805]
  [0.5751]
  [0.9919]
  [0.9662]
  [0.9445]
  [0.8228]]

 [[0.7196]
  [0.7694]
  [0.9058]
  [0.9928]
  [0.6395]
  [0.7883]
  [0.9662]
  [0.9796]
  [0.5421]
  [0.6975]
  [0.6471]
  [0.9699]
  [0.879 ]
  [0.8866]
  [0.9107]
  [0.9457]
  [0.725 ]
  [0.8705]
  [0.9051]
  [0.3331]
  [0.8294]
  [0.792 ]
  [0.913 ]
  [0.9682]
  [0.988 ]
  [0.6424]
  [0.9263]
  [0.8372]
  [0.7571]
  [0.986 ]
  [0.894 ]
  [0.971

In [78]:
batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

print('class          shape: ', pred_classes.shape)
print(pred_classes.eval())
print('roi_grid shape',tf.shape(roi_grid).eval())
print(roi_grid.eval())
print('batch_grid     shape',tf.shape(batch_grid).eval())
print(batch_grid.eval())


bbox_idx        = tf.to_float(tf.expand_dims(roi_grid, axis = -1))
scatter_classes = tf.stack([batch_grid , pred_classes, roi_grid],axis = -1)


print('-- stack results ----')
print('scatter_classes', type(scatter_classes), 'shape',tf.shape(scatter_classes).eval())
print(scatter_classes.eval())

class          shape:  (3, 32)
[[0 0 3 0 1 1 2 2 1 0 1 0 1 1 3 0 3 2 2 2 2 0 3 0 2 3 2 2 0 0 0 3]
 [1 1 3 3 0 3 2 0 3 3 3 3 3 3 1 3 0 2 3 1 0 0 1 2 0 0 3 3 0 0 1 3]
 [1 3 3 0 3 0 2 3 1 0 0 1 1 1 3 3 2 3 0 3 3 2 2 0 3 2 3 3 0 2 1 0]]
roi_grid shape [ 3 32]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
batch_grid     shape [ 3 32]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]
-- stack results ----
scatter_inds <class 'tensorflow.python.framework.ops.Tensor'> shape [ 3 32  3]
[[[ 0  0  0]
  [ 0  0  1]
  [ 0  3  2]
  [ 0  0  3]
  [ 0  1  4]
  [ 0  1  5]
  [ 0  2  6]
  [ 0  2  7]
  [ 0  1  

###  Build indicies to gather bounding boxes from bboxes_4d corrsponding to predicted class
#### Only used if we want to use mrcnn_bboxes (batch_size, num_rois, num_classes, 4)

batch_size x nuum_detections x 4

In [80]:
# gather_boxes = tf.stack([batch_grid, roi_grid, pred_classes, ], axis = -1)

# print('-- gather_boxes  ----')
# print('gather_boxes inds', type(gather_boxes), 'shape',tf.shape(gather_boxes).eval())
# print(gather_boxes.eval())

# mrcnn_bboxes_selected = tf.gather_nd(mrcnn_bboxes, gather_boxes)
# print(' padding required for output_rois : ', mrcnn_bboxes_selected.get_shape())
# print(mrcnn_bboxes_selected[0].eval())

print(' output_rois shape ', output_rois.get_shape())
print(pred_classes[0].eval())
print(output_rois[0].eval())



###  Generate tensor of score - bbox - class 

In [14]:
pred_array = tf.concat([ bbox_idx, pred_scores , output_rois, pred_classes_exp], axis=-1)

print(' resulting tensor : pred_array',  type(pred_array), pred_array.shape)
print(pred_array[0].eval())
print('------')
print(pred_array[1].eval())
print('------')
print(pred_array[2].eval())

 resulting tensor : a_boxes_3d <class 'tensorflow.python.framework.ops.Tensor'> (3, 32, 7)
[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
 [  1.       0.9139  38.2599 103.6086  84.2878  61.6374   0.    ]
 [  2.       0.866   45.7513  24.8754  24.5708  37.1656   3.    ]
 [  3.       0.5591 106.9807 118.7934 106.3466   7.4645   0.    ]
 [  4.       0.8094  61.6912   3.5655 122.1108 118.7579   1.    ]
 [  5.       0.591   77.9712  18.8217  15.8669  12.7541   1.    ]
 [  6.       0.9869  46.8547  69.4961  99.9029  86.3197   2.    ]
 [  7.       0.7943 120.8985  56.4598  65.6325  11.3239   2.    ]
 [  8.       0.9574  71.3561  98.4842  67.1786  56.1681   1.    ]
 [  9.       0.8659  81.8618  17.9545  60.2488  71.9641   0.    ]
 [ 10.       0.4301  32.2131  71.4339  22.2361  89.2849   1.    ]
 [ 11.       0.3574  55.5344  25.4976  58.2088 112.5119   0.    ]
 [ 12.       0.9281  66.7568  56.2779 111.4212  49.245    1.    ]
 [ 13.       0.9126 101.3707  95.1844   1.9615  96.

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd_update

    Output is converted to NUMN_BATCHES x NUM_CLASSES x num_rois x columns 

In [16]:
# scatt1 = tf.scatter_nd(scatter_inds, pred_array, [batch_size, num_classes, num_rois])
# print('scatt1  shape ', scatt1.get_shape(), scatt)

zeros = tf.zeros([batch_size,num_classes, num_rois,7], dtype=tf.float32)
ref   = tf.Variable(zeros)
init = tf.global_variables_initializer().run()

pred_scatt = tf.scatter_nd_update(ref, scatter_classes, pred_array)


print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
print(pred_scatt[0].eval())
print('------------')
print(pred_scatt[1].eval())
print('------------')
print(pred_scatt[2].eval())



scatter shape is  (3, 4, 32, 7) Tensor("ScatterNdUpdate:0", shape=(3, 4, 32, 7), dtype=float32_ref)
[[[  0.       0.9825  16.8633  29.8584 100.7555  26.0595   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  3.       0.7143  68.6662   6.0315  60.8129  57.7871   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  9.       0.9014  68.3501 116.1659 107.5235  28.9433   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [ 12.       0.6446  49.6512 126.1225  34.0192  72.4129   0.    ]
  [  0.       0.       0.    

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd

    Output is converted to NUMN_BATCHES x NUM_CLASSES x NUM_ROIS x columns(7)

In [96]:
pred_scatt2 = tf.scatter_nd(scatter_classes, pred_array, [batch_size, num_classes, num_rois,7])
# tst = ref
print('scatter shape is ', ref.get_shape(), ref)
print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
print('scatter shape is ', pred_scatt2.get_shape(), pred_scatt2)
# ref[0,0,0,0] = tf.assign(ref[0,0,0,0] , + 1)
# 
# print(ref[0,0,:4].eval())
print(pred_scatt[0,0,:4].eval())
print(pred_scatt2[0,0,:4].eval())
# print('------------')
# print(pred_scatt2[1].eval())
# print('------------')
# print(pred_scatt2[2].eval())

equality = tf.equal(pred_scatt[0,0,0], pred_scatt2[0,0,0])
print(tf.reduce_all(equality).eval())

NameError: name 'pred_array' is not defined

###  Sort each slice based on score (column 1)

###  Rearrange each slice based on sorted indices in "indcs"

In [183]:

sort_vals, sort_inds = tf.nn.top_k(pred_scatt[:,:,:,1], k=pred_scatt.shape[2])

# print(tf.shape(sort_vals).eval())
# print(sort_vals.eval())
print()
print(tf.shape(sort_inds).eval())
print(sort_inds.eval())


class_grid, batch_grid, dtct_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_rois))


# print('class_grid', type(class_grid), 'shape',tf.shape(class_grid).eval())
# print(class_grid.eval())

# print('batch_grid', type(batch_grid), 'shape',tf.shape(batch_grid).eval())
# print(batch_grid.eval())

# print('class_grid', type(dtct_grid), 'shape',tf.shape(dtct_grid).eval())
# print(dtct_grid.eval())


gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)


print('-- stack results ----')
print('gather_inds', type(gather_inds), 'shape',tf.shape(gather_inds).eval())
print(gather_inds.eval())

pred_tensor = tf.gather_nd(pred_scatt, gather_inds)

print()
print('-- gather_nd results (A-boxes sorted by score ----')
print(' srtd_boxes shape : ', tf.shape(pred_tensor).eval())
print(pred_tensor.eval())




[ 3  4 28]
[[[ 6  8  3 10  7  9  1  0  2  4  5 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
  [ 5 12 14  4  0  1  2  3  6  7  8  9 10 11 13 15 16 17 18 19 20 21 22 23 24 25 26 27]
  [13  2  0 11  1  3  4  5  6  7  8  9 10 12 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
  [15 16  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 17 18 19 20 21 22 23 24 25 26 27]]

 [[11 12 14 15  0 10  1  2  3  4  5  6  7  8  9 13 16 17 18 19 20 21 22 23 24 25 26 27]
  [ 7  4  0  1  2  3  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
  [ 3  8  1  9  2 13 16  0  4  5  6  7 10 11 12 14 15 17 18 19 20 21 22 23 24 25 26 27]
  [ 6  5  0  1  2  3  4  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]]

 [[ 6 10  5  0  1  2  3  4  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27]
  [16 12  8  0  1  2  3  4  5  6  7  9 10 11 13 14 15 17 18 19 20 21 22 23 24 25 26 27]
  [ 1  2  4 15 11  0  3 13  5  6  7  8  9 10 12 14 16 17 18 19 20 21 22 23 24 25 26 27]
  [ 9 14  7  0  

###  Display for visual check

In [212]:
print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
img  = 1
print(pred_scatt[img,0].eval())
print(pred_tensor[img,0].eval())

print('------------')
print(pred_scatt[img, 1].eval())
print(pred_tensor[img,1].eval())

print('------------')
print(pred_scatt[img,2].eval())
print(pred_tensor[img, 2].eval())

print('------------')
print(pred_scatt[img,3].eval())
print(pred_tensor[img, 3].eval())


scatter shape is  (3, 4, 28, 7) Tensor("ScatterNdUpdate_49:0", shape=(3, 4, 28, 7), dtype=float32_ref)
[[  0.       0.9892  97.5308  95.3976  45.0381  93.3439   0.    ]
 [  1.       0.6332  99.8659  94.113   98.2793 113.5731   0.    ]
 [  2.       0.9479  25.3408  69.9052  28.2969  81.1685   0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  6.       0.9542  65.458  121.6128  47.0561  39.0045   0.    ]
 [  7.       0.9981  37.6414 121.3623  56.5611 112.1401   0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [ 11.       0.7284   8.2115 126.8421  98.9321  96.1301   0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.      

[[  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [ 12.       0.7761  17.571    3.0607 103.6246 103.396    3.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [ 14.       0.5678  36.9084  48.5982  69.6596  90.221    3.    ]
 [  0.    

In [95]:
tf.count_nonzero()

AttributeError: module 'tensorflow' has no attribute 'count_non_zero'

In [213]:
a00 = pred_tensor
print(a00[:,:,:,0].get_shape())

a00_cnt = tf.count_nonzero(a00[:,:,:,0],axis = -1)
print(a00_cnt.get_shape())
a00_cnt.eval()

(3, 4, 28)
(3, 4)


array([[16,  3,  5,  3],
       [16,  7,  2,  2],
       [14,  5,  5,  3]], dtype=int64)

In [208]:
a00 = pred_tensor[0]
print(a00[:,:,0].get_shape())

a00_cnt = tf.count_nonzero(a00[:,:,0],axis = 1)
print(a00_cnt.get_shape())
a00_cnt.eval()

(4,)


array([16,  3,  5,  3], dtype=int64)

###  Sort each slice based on class id (obsolete)

#### Now we split based on class id first then sort based on score - no need to sort on class beforehand

In [11]:
# sort_vals, sort_inds = tf.nn.top_k(-pred_array[:,:,6], k=pred_array.shape[1])

# print(tf.shape(sort_vals).eval())
# print(sort_vals.eval())
# print()
# print(tf.shape(sort_inds).eval())
# print(sort_inds.eval())

###  Scatter to class layers - Loop / Dynamic Partition method  (Obsolete)
 
We are scattering using the scatter_nd_update method

In [15]:
print('shape of srtd_boxes ', srtd_boxes.shape)
partition_inds = tf.to_int32(srtd_boxes[...,6])
print(partition_inds.eval())


batch_stack = []
for i in range(batch_size):
    
    splitted = tf.dynamic_partition(srtd_boxes[i], partition_inds[i], num_classes)

    for idx, split in enumerate(splitted):
#         print(' item ', idx,' shape ',split.shape, 'shape:', tf.shape(split).eval(), 'rank: ', tf.rank(split).eval())
        padding = num_rois - tf.shape(split)[0]
        split = tf.pad(split,[[0,padding],[0,0]]) 
#         print(' num of padding rows tt add ', padding.eval())        
#         print(' shape after padding : ', tf.shape(split).eval())
#         print(split.eval())
        splitted[idx] = split
    stacked = tf.stack(splitted, axis = 0)
    batch_stack.append(stacked)

batch_tensor = tf.stack(batch_stack, axis = 0)
print(' batch _tensor shape is ', tf.shape(batch_tensor).eval())
print(batch_tensor[0].eval())    


shape of srtd_boxes  (3, 28, ?)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 2 2 2 2 2 2 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 3 3 3 3 3 3 3 3 3 3 3]]
 batch _tensor shape is  [ 3  4 28  7]
[[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
  [  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
  [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
  [  9.       0.8659 110.8402  12.9817  28.9916   4.0611   0.    ]
  [ 11.       0.3574  45.7513  24.8754  24.5708  37.1656   0.    ]
  [ 15.       0.7801  99.8521  70.4273  73.7088  58.9241   0.    ]
  [ 17.       0.      61.6912   3.5655 122.1108 118.7579   0.    ]
  [ 18.       0.     102.1897  90.9454 105.3337  90.0073   0.    ]
  [ 19.       0.      53.3086  12.4369 117.3345  58.387    0.    ]
  [ 20.       0.      21.658   39.0892  78.8661  66.4719   0.    ]
  [ 21.       0.      77.9712  18.8217  15.8669  12.7541   0.    ]
  [ 22.       

## Experimental Code

###  Experiment for sort ordering 

In [462]:
indices = tf.constant([
            [[0,2],[0,1],[0,0]],
            [[1,2],[1,1],[1,0]],
            [[2,0],[2,1],[2,1]],
          ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03'], 
           ['0-10', '0-11', '0-12', '0-13'],
           ['0-20', '0-21', '0-22', '0-23'],
           ['0-30', '0-31', '0-32', '0-33']],
    
          [['1-00', '1-01', '1-02', '1-03'], 
           ['1-10', '1-11', '1-12', '1-13'],
           ['1-20', '1-21', '1-22', '1-23'],
           ['1-30', '1-31', '1-32', '1-33']],
    
          [['2-00', '2-01', '2-02', '2-03'], 
           ['2-10', '2-11', '2-12', '2-13'],
           ['2-20', '2-21', '2-22', '2-23'],
           ['2-30', '2-31', '2-32', '2-33']]    
         ])
print(params.shape, '   ', indices.shape)
res = tf.gather_nd(params, indices)
res.eval()



(3, 4, 4)     (3, 3, 2)


array([[[b'0-20', b'0-21', b'0-22', b'0-23'],
        [b'0-10', b'0-11', b'0-12', b'0-13'],
        [b'0-00', b'0-01', b'0-02', b'0-03']],

       [[b'1-20', b'1-21', b'1-22', b'1-23'],
        [b'1-10', b'1-11', b'1-12', b'1-13'],
        [b'1-00', b'1-01', b'1-02', b'1-03']],

       [[b'2-00', b'2-01', b'2-02', b'2-03'],
        [b'2-10', b'2-11', b'2-12', b'2-13'],
        [b'2-10', b'2-11', b'2-12', b'2-13']]], dtype=object)

###  Experiment with GATHER() for splitting into classes

In [84]:
# indices = tf.constant([
#             [0,2,2],[0,1,1],[0,0,0],
#             [1,2,2],[1,1,1],[1,0,0],
#             [2,0,0],[2,1,1],[2,1,1],
#            ])
# indices = tf.constant([ [0,2],[0,1],[0,0],[1,2],[1,1],[1,0],[1,2],[1,3], [2,3]] ) # 9 x 2
indices = tf.constant([
           [
            [[0,2],[0,1],[0,0],[1,2],[2,3]],
            [[1,1],[1,0],[1,2],[1,3],[1,1]]
           ]
           ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03', '0-04', '0-05', '0-06'], 
           ['0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16'],
           ['0-20', '0-21', '0-22', '0-23', '0-24', '0-25', '0-26'],
           ['0-30', '0-31', '0-32', '0-33', '0-34', '0-35', '0-36'],
           ['0-40', '0-41', '0-42', '0-43', '0-44', '0-45', '0-46']],
                                                          
          [['1-00', '1-01', '1-02', '1-03', '1-04', '1-05', '1-06'], 
           ['1-10', '1-11', '1-12', '1-13', '1-14', '1-15', '1-16'],
           ['1-20', '1-21', '1-22', '1-23', '1-24', '1-25', '1-26'],
           ['1-30', '1-31', '1-32', '1-33', '1-34', '1-35', '1-36'],
           ['1-40', '1-41', '1-42', '1-43', '1-44', '1-45', '1-46']],
                                                          
          [['2-00', '2-01', '2-02', '2-03', '2-04', '2-05', '2-06'], 
           ['2-10', '2-11', '2-12', '2-13', '2-14', '2-15', '2-16'],
           ['2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26'],
           ['2-30', '2-31', '2-32', '2-33', '2-34', '2-35', '2-36'],
           ['2-40', '2-41', '2-42', '2-43', '2-44', '2-45', '2-46']]    
         ])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[:0], params.shape[:1], params.shape[:2], params.shape[:3])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[0:], params.shape[1:], params.shape[2:], params.shape[3:])
print(' indices.shape  ', indices.shape)
res = tf.gather_nd(params, indices)
print('result shape   ', res.get_shape())
res.eval()

 params sahape:  (3, 5, 7)  parms[:axis] () (3,) (3, 5) (3, 5, 7)
 params sahape:  (3, 5, 7)  parms[:axis] (3, 5, 7) (5, 7) (7,) ()
 indices.shape   (1, 2, 5, 2)
result shape    (1, 2, 5, 7)


array([[[[b'0-20', b'0-21', b'0-22', b'0-23', b'0-24', b'0-25', b'0-26'],
         [b'0-10', b'0-11', b'0-12', b'0-13', b'0-14', b'0-15', b'0-16'],
         [b'0-00', b'0-01', b'0-02', b'0-03', b'0-04', b'0-05', b'0-06'],
         [b'1-20', b'1-21', b'1-22', b'1-23', b'1-24', b'1-25', b'1-26'],
         [b'2-30', b'2-31', b'2-32', b'2-33', b'2-34', b'2-35', b'2-36']],

        [[b'1-10', b'1-11', b'1-12', b'1-13', b'1-14', b'1-15', b'1-16'],
         [b'1-00', b'1-01', b'1-02', b'1-03', b'1-04', b'1-05', b'1-06'],
         [b'1-20', b'1-21', b'1-22', b'1-23', b'1-24', b'1-25', b'1-26'],
         [b'1-30', b'1-31', b'1-32', b'1-33', b'1-34', b'1-35', b'1-36'],
         [b'1-10', b'1-11', b'1-12', b'1-13', b'1-14', b'1-15', b'1-16']]]], dtype=object)

###  Experiment with SCATTER() for splitting into classes

In [83]:
# del indices, updates, shape

indices =  tf.constant([
                         #|
                         #| 
                        [[0,0,0], [0,3,0], [0,2,0], [0,1,0],[0,1,1],[0,1,2],[0,2,1],[0,3,2] ],
                        [[1,0,0], [1,3,0], [1,2,0], [1,1,0],[1,1,1],[1,1,2],[1,2,1],[1,3,2] ],
                        [[2,0,0], [2,3,0], [2,2,0], [2,1,0],[2,1,1],[2,1,2],[2,2,1],[2,3,2] ]
])
                            #<--- each one corrsponds to an entry in  the updates array [row , position in resulting array]
    
#                        [[1,1], [3,1], [2,1], [1,1],[1,1],[1,2],[2,1],[3,2] ],
#                        [[0,0], [3,0], [2,0], [1,0],[1,1],[1,2],[2,1],[3,2] ]
    
###----------------------------------------------------
### Our bounding box array
###---------------------------------------------------
updates = tf.constant( [
          [[1000, 1001, 1002, 1003, 1004, 1005, 1006], 
           [1010, 1011, 1012, 1013, 1014, 1015, 1016],
           [1020, 1021, 1022, 1023, 1024, 1025, 1026],
           [1030, 1031, 1032, 1033, 1034, 1035, 1036],
           [1040, 1041, 1042, 1043, 1044, 1045, 1046],
           [1050, 1051, 1052, 1053, 1054, 1055, 1056],
           [1060, 1061, 1062, 1063, 1064, 1065, 1066],
           [1070, 1071, 1072, 1073, 1074, 1075, 1076] ],
          
          [[2000, 2001, 2002, 2003, 2004, 2005, 2006], 
           [2010, 2011, 2012, 2013, 2014, 2015, 2016],
           [2020, 2021, 2022, 2023, 2024, 2025, 2026],
           [2030, 2031, 2032, 2033, 2034, 2035, 2036],
           [2040, 2041, 2042, 2043, 2044, 2045, 2046],
           [2050, 2051, 2052, 2053, 2054, 2055, 2056],
           [2060, 2061, 2062, 2063, 2064, 2065, 2066],
           [2070, 2071, 2072, 2073, 2074, 2075, 2076]] , 

          [[3000, 3001, 3002, 3003, 3004, 3005, 3006], 
           [3010, 3011, 3012, 3013, 3014, 3015, 3016],
           [3020, 3021, 3022, 3023, 3024, 3025, 3026],
           [3030, 3031, 3032, 3033, 3034, 3035, 3036],
           [3040, 3041, 3042, 3043, 3044, 3045, 3046],
           [3050, 3051, 3052, 3053, 3054, 3055, 3056],
           [3060, 3061, 3062, 3063, 3064, 3065, 3066],
           [3070, 3071, 3072, 3073, 3074, 3075, 3076] ]    
         ])                       

zeros = tf.zeros([3,4,8,7], dtype=tf.int32)
ref   = tf.Variable(zeros)
scatt1= tf.Variable(zeros)
shape = tf.constant([3,4, 8, 7])
 
# init = tf.global_variables_initializer().run()
# print('indices shape ', indices.shape,  'indices_shape [-1]', indices.shape[-1],  'indices_shape [:-1]', indices.shape[:-1])
# print('updates.shape ', updates.shape)
# print('ref     shape ', ref.get_shape(), ref)
# print('scatt1  shape ', scatt1.get_shape(), scatt)

scatt1 = tf.scatter_nd(indices, updates, shape)
# print('scatt1  shape ', scatt1.get_shape(), scatt)
# print(scatt1[0].eval())
# print('------------')
# print(scatt1[1].eval())
# print('------------')
# print(scatt1[2].eval())

# scatt = tf.scatter_nd_update(ref, indices, updates)
# print('scatter shape is ', scatt.get_shape(), scatt)
# print(scatt.eval())

print()


indices shape  (3, 8, 3) indices_shape [-1] 3 indices_shape [:-1] (3, 8)
updates.shape  (3, 8, 7)
ref     shape  (3, 4, 8, 7) <tf.Variable 'Variable_113:0' shape=(3, 4, 8, 7) dtype=int32_ref>
scatt1  shape  (3, 4, 8, 7) <tf.Variable 'Variable_102:0' shape=(3, 4, 8, 7) dtype=int32_ref>
scatt1  shape  (3, 4, 8, 7) <tf.Variable 'Variable_102:0' shape=(3, 4, 8, 7) dtype=int32_ref>
[[[1000 1001 1002 1003 1004 1005 1006]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]]

 [[1030 1031 1032 1033 1034 1035 1036]
  [1040 1041 1042 1043 1044 1045 1046]
  [1050 1051 1052 1053 1054 1055 1056]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0 

In [49]:
indices = tf.constant([
                        [[0,0], [3,0], [2,0], [1,0]]
#                     ,[[1,3], [1,3], [1,2], [1,0]]
                      ])
updates = tf.constant([
                       [[5, 5, 5, 5, 0], 
                        [6, 6, 6, 6, 3],
                        [7, 7, 7, 7, 2], 
                        [8, 8, 8, 8, 1]]
                       
#                      , [[5, 5, 5, 3], 
#                         [6, 6, 6, 3],
#                         [7, 7, 7, 2], 
#                         [8, 8, 8, 0]]
                       ])
shape = tf.constant([4, 4, 5])
print(indices.shape, 'inputs shape', updates.shape, shape.shape)
scatter = tf.scatter_nd(indices, updates, shape)
print('scatter shape is ', scatter.get_shape())
print(scatter.eval())

(1, 4, 2) inputs shape (1, 4, 5) (3,)
scatter shape is  (4, 4, 5)
[[[5 5 5 5 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[8 8 8 8 1]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[7 7 7 7 2]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[6 6 6 6 3]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [70]:
num_detections = 8
# batch_size is  3
gt_class_ids = tf.constant([[1,2,3,0,0,0,0,0],[3,3,3,0,0,0,0,0],[1,2,2,0,0,2,0,0]])
gt_bboxes    = tf.random_uniform([batch_size,3,4], maxval = 127, dtype=tf.int32)


gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
print('    gt_classes_exp: ' ,gt_classes_exp.get_shape())
print(gt_classes_exp.eval())


zeros        = tf.zeros([batch_size,5,4], dtype = tf.int32)
gt_bboxes    = tf.concat([gt_bboxes, zeros], axis = 1)
gt_bboxes    = tf.to_float(gt_bboxes)
print('\n    gt_bboxes: ' ,gt_bboxes.get_shape())
print(gt_bboxes.eval())

mask = tf.greater(gt_class_ids,0)
print(mask.eval())

gt_scores     = tf.where(mask, tf.ones_like(gt_class_ids), tf.zeros_like(gt_class_ids))
gt_scores_exp = tf.to_float(tf.expand_dims(gt_scores, axis=-1))
print('\n    gt_scores ', gt_scores_exp.get_shape())
print(gt_scores_exp.eval())

batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size    , dtype=tf.int32), 
                                    tf.range(num_detections , dtype=tf.int32), indexing = 'ij' )

print('\n    bbox_grid   ', type(bbox_grid)  , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())
print('\n    batch_grid ', type(batch_grid), 'shape', batch_grid.get_shape())
print(batch_grid.eval())
 
bbox_idx_zeros  = tf.zeros_like(bbox_grid)
bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
print('    bbox_idx', type(bbox_idx), 'shape', bbox_idx.get_shape())
print(bbox_idx.eval())

    gt_classes_exp:  (3, 8, 1)
[[[1.]
  [2.]
  [3.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[3.]
  [3.]
  [3.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [2.]
  [2.]
  [0.]
  [0.]
  [2.]
  [0.]
  [0.]]]

    gt_bboxes:  (3, 8, 4)
[[[ 90. 123.  82.  90.]
  [ 98.  49.  65. 110.]
  [ 93.  21.  66.  21.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]

 [[ 48.  13. 109. 125.]
  [108.  25.  33.  18.]
  [120.  62.  35. 102.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]

 [[124.  52.  83.  95.]
  [117.  21. 109.  57.]
  [114.  71.  59.  73.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]]
[[ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False  True False False]]

    gt_scores  (3, 8, 1)
[[[1.]

In [71]:
gt_array        = tf.concat([bbox_idx, gt_scores_exp , gt_bboxes, gt_classes_exp], axis=2)
print('    gt_array  ',  type(gt_array), gt_array.shape)
print(gt_array.eval())


    gt_array   <class 'tensorflow.python.framework.ops.Tensor'> (3, 8, 7)
[[[  0.   1.  96.  60.  55. 116.   1.]
  [  1.   1. 116.  95.  45.  99.   2.]
  [  2.   1. 119.  18.  61. 104.   3.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]]

 [[  0.   1.  69.  61.  34.  33.   3.]
  [  1.   1. 106.  82.  38.  77.   3.]
  [  2.   1.  36.   4. 115.  77.   3.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]]

 [[  0.   1.   1. 115.  90.  29.   1.]
  [  1.   1.  75.  66.  49.  56.   2.]
  [  2.   1.  95.  91.  21. 114.   2.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  5.   1.   0.   0.   0.   0.   2.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   

In [72]:
scatter_classes = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
print('\n    -- stack results ----')
print('\n    scatter_classes', type(scatter_classes), 'shape',tf.shape(scatter_classes).eval())
print(scatter_classes.eval())


    -- stack results ----

    scatter_classes <class 'tensorflow.python.framework.ops.Tensor'> shape [3 8 3]
[[[0 1 0]
  [0 2 1]
  [0 3 2]
  [0 0 3]
  [0 0 4]
  [0 0 5]
  [0 0 6]
  [0 0 7]]

 [[1 3 0]
  [1 3 1]
  [1 3 2]
  [1 0 3]
  [1 0 4]
  [1 0 5]
  [1 0 6]
  [1 0 7]]

 [[2 1 0]
  [2 2 1]
  [2 2 2]
  [2 0 3]
  [2 0 4]
  [2 2 5]
  [2 0 6]
  [2 0 7]]]


In [63]:
gt_scatter = tf.scatter_nd(scatter_classes, gt_array, [batch_size, num_classes, num_detections,7])
print('    gt_tensor shape is ', gt_scatter.get_shape(), gt_scatter)
gt_scatter.eval()

    gt_tensor shape is  (3, 4, 8, 7) Tensor("ScatterNd_2:0", shape=(3, 4, 8, 7), dtype=float32)


array([[[[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.]],

        [[  0.,   1.,  47., 102.,  91.,  30.,   1.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.]],

        [[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  1.,   1.,  54., 109., 100.,  68.,   2.],
         [  0.,   0.,   0.,   0.,   0.,   

In [77]:
sort_vals, sort_inds = tf.nn.top_k(gt_scatter[:,:,:,0], k=gt_scatter.shape[2])
print('    sort vals shape : ', sort_vals.get_shape())
print(sort_vals.eval())
print('    sort inds shape : ', sort_inds.get_shape())
print(sort_inds.eval())

    sort vals shape :  (3, 4, 8)
[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [2. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [2. 1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [5. 2. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]
    sort inds shape :  (3, 4, 8)
[[[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [1 0 2 3 4 5 6 7]
  [2 0 1 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [2 1 0 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [5 2 1 0 3 4 6 7]
  [0 1 2 3 4 5 6 7]]]


In [73]:
# build gathering indexes to use in sorting 
class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_detections))

print('    class_grid  ', type(class_grid) , 'shape', class_grid.get_shape())
print(class_grid.eval())
print('    batch_grid  ', type(batch_grid) , 'shape', batch_grid.get_shape())
print(class_grid.eval())
print('    bbox_grid    ', type(bbox_grid) , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())

    class_grid   <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]]
    batch_grid   <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]]
    bbox_grid     <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]]


In [75]:
gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    gatehr_inds ', gather_inds.get_shape())
print(gather_inds.eval())

gt_tensor = tf.gather_nd(gt_scatter, gather_inds)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    pred_tensor ', gt_tensor.get_shape())
print(gt_tensor.eval())

    -- pred_tensor results (A-boxes sorted by score ----
    gatehr_inds  (3, 4, 8, 3)
[[[[0 0 0]
   [0 0 1]
   [0 0 2]
   [0 0 3]
   [0 0 4]
   [0 0 5]
   [0 0 6]
   [0 0 7]]

  [[0 1 0]
   [0 1 1]
   [0 1 2]
   [0 1 3]
   [0 1 4]
   [0 1 5]
   [0 1 6]
   [0 1 7]]

  [[0 2 1]
   [0 2 0]
   [0 2 2]
   [0 2 3]
   [0 2 4]
   [0 2 5]
   [0 2 6]
   [0 2 7]]

  [[0 3 2]
   [0 3 0]
   [0 3 1]
   [0 3 3]
   [0 3 4]
   [0 3 5]
   [0 3 6]
   [0 3 7]]]


 [[[1 0 0]
   [1 0 1]
   [1 0 2]
   [1 0 3]
   [1 0 4]
   [1 0 5]
   [1 0 6]
   [1 0 7]]

  [[1 1 0]
   [1 1 1]
   [1 1 2]
   [1 1 3]
   [1 1 4]
   [1 1 5]
   [1 1 6]
   [1 1 7]]

  [[1 2 0]
   [1 2 1]
   [1 2 2]
   [1 2 3]
   [1 2 4]
   [1 2 5]
   [1 2 6]
   [1 2 7]]

  [[1 3 2]
   [1 3 1]
   [1 3 0]
   [1 3 3]
   [1 3 4]
   [1 3 5]
   [1 3 6]
   [1 3 7]]]


 [[[2 0 0]
   [2 0 1]
   [2 0 2]
   [2 0 3]
   [2 0 4]
   [2 0 5]
   [2 0 6]
   [2 0 7]]

  [[2 1 0]
   [2 1 1]
   [2 1 2]
   [2 1 3]
   [2 1 4]
   [2 1 5]
   [2 1 6]
   [2 1 7]]

  [[2 2 5